In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


from pyspark import SparkContext
import pyspark.sql.functions

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.mllib.evaluation import RankingMetrics

sc = SparkContext.getOrCreate()
#spark = pyspark.sql.SparkSession


In [2]:
spark, sc

(<pyspark.sql.session.SparkSession at 0x7f450d409110>,
 <pyspark.context.SparkContext at 0x7f4530889590>)

In [2]:
# Read in the ratings data for training set
train = sc.textFile("data/new_subset_data/train_commits.csv")
# Filter out first column of the header
train_data = train.filter(lambda l: not l.startswith('user_id'))
                          
# Read in the ratings data for testing set
test = sc.textFile("data/new_subset_data/test_commits.csv")
# Filter out first column of the header
test_data = test.filter(lambda l: not l.startswith('user_id'))

# Read in the ratings data for entire set
entire = sc.textFile("data/new_subset_data/entire_commits.csv")
# Filter out first column of the header
entire_data = entire.filter(lambda l: not l.startswith('user_id'))


def parseLine(line, sep=','):
    fields = line.split(sep)
    return Rating(user=int(fields[0]), product=int(fields[1]), rating=float(fields[2]))

 
train_set = train_data.map(lambda r: parseLine(r))
test_set = test_data.map(lambda r: parseLine(r))
entire_set = entire_data.map(lambda r: parseLine(r))

In [ ]:
entire_set.count()

In [3]:
# Train a model on to predict user-product ratings
model = ALS.trainImplicit(entire_set, 
                          rank=50, 
                          iterations=20, 
                          lambda_=10.0, 
                          alpha=20.0)

In [7]:
model.save(sc, './models/Commits_Model')

Py4JJavaError: An error occurred while calling o80.save.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/home/ubuntu/PROJECT/github-collaborator/models/Commits_Model/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1191)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1168)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1168)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:362)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1168)
	at org.apache.spark.rdd.PairRDDFunctions$$anonf

### Try saving a small test model

In [ ]:
mini_test_set = sc.parallelize(entire_set.take(100))

In [ ]:
# Mini test model 
mini_test = ALS.trainImplicit(mini_test_set, 
                          rank=50, 
                          iterations=20, 
                          lambda_=10.0, 
                          alpha=20.0)

In [ ]:
mini_test.save(sc, "./models/mini_test")

In [ ]:
!rm -r 'models/mini_test/'

In [ ]:
test_model = MatrixFactorizationModel.load(sc, "./models/mini_test")

In [ ]:
mini_test.userFeatures().collect()

## Make predicitons

In [ ]:
# Get predicted ratings on all user-products fed in
testData = test_set.map(lambda p: (p.user, p.product), p.rating)
trainData = train_set.map(lambda p: (p.user, p.product), p.rating)
#entireData = entire_set.map(lambda p: (p.user, p.product), p.rating)

predictions = model.predictAll(testData).map(lambda r: (r.user, r.product), r.rating)
#predictions_entire = model.predictAll(entireData).map(lambda r: (r.user, r.product), r.rating)
predictions.take(10)

In [ ]:
# Read in the user-repo lookup table
users_repos_RDD = sc.textFile("data/user_repo_lookup.csv") \
                    .map(lambda line: line.split("\t"))
# Filter out first column of the header
user_repo_lookup = users_repos_RDD.filter(lambda l: not l.startswith('user_id'))


In [4]:
users_repos = pd.read_csv('data/user_repo_lookup.csv', sep='\t')
users_repos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59366945 entries, 0 to 59366944
Data columns (total 6 columns):
user_id        int64
login          object
repo_id        int64
forked_from    object
repo_name      object
url            object
dtypes: int64(2), object(4)
memory usage: 2.7+ GB


In [5]:
pd.set_option('display.max_colwidth', -1)

In [6]:
# User predictions as repo ids
def get_recomm_repo_ids(model, userID, k=50):
    predictions = model.recommendProducts(userID, k)
    return [predictions[i].product for i in range(len(predictions))]

In [7]:
# Urls for user recommendations
def get_recomm_repo_urls(lookupdf, repo_ids):
    return lookupdf[lookupdf['repo_id'].isin(repo_ids)]


# Linus Torvalds recos

#### If you're really famous in the GitHub world and have tons of forks of your projects, the recommender just returns all the forks of your own project...

In [ ]:
repo_ids = get_recomm_repo_ids(model, 5203, 25)

In [ ]:
torvalds_recos = get_recomm_repo_urls(users_repos, repo_ids)
torvalds_recos.to_json('recos_json/5203_torvalds')
torvalds_recos.to_csv('recos_csv/5203_torvalds', sep='\t', index=False)

In [ ]:
users_repos[users_repos['login'] == 'torvalds']

# Matt (galvanze) recos

In [ ]:
repo_ids = get_recomm_repo_ids(model, 31533621, 25)

In [ ]:
get_recomm_repo_urls(users_repos, repo_ids)

In [ ]:
users_repos[users_repos['login'] == 'madrury-galvanize']

# Matt's recos

In [ ]:
repo_ids = get_recomm_repo_ids(model, 1795649, 25)

In [ ]:
get_recomm_repo_urls(users_repos, repo_ids)
madrury = get_recomm_repo_urls(users_repos, repo_ids)
madrury.to_json('recos_json/1795649_madrury')
madrury.to_csv('recos_csv/1795649_madrury', sep='\t', index=False)

In [ ]:
users_repos[users_repos['login'] == 'madrury']

# John's recos 

In [ ]:
repo_ids = get_recomm_repo_ids(model, 8411950, 25)

In [ ]:
johink = get_recomm_repo_urls(users_repos, repo_ids)

johink.to_json('recos_json/8411950_johink')
madrury.to_csv('recos_csv/8411950_johink', sep='\t', index=False)

In [ ]:
users_repos[users_repos['login'] == 'johink']

## All user recos output to MySQL db


#### Install dependencies
`sudo apt-get install python-pip python-dev libmysqlclient-dev`  
#### then install MySQLdb  
`pip install MySQL-python`  

other docs:
- http://docs.sqlalchemy.org/en/latest/core/engines.html
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html

In [8]:
from pandas.io import sql
import pymysql
from sqlalchemy import create_engine
# conn = pymysql.connect(host='localhost', port=3306, user='root', passwd='0', db='user_recs')
engine = create_engine('mysql://root:0@localhost/user_recs')


# from pandas.io import sql
# import MySQLdb
# conn = MySQLdb.connect()

# import _mysql
# db=_mysql.connect(host="localhost",user="root",
#                   passwd="0",db="github")

# import MySQLdb
# db=MySQLdb.connect(host="localhost",user="root",
#                    passwd="0",db="user_recs")

In [ ]:
# Make Top 50 repo rec for every user and append to MySQL table
num_users_not_in_model = 0
unique_users = set(users_repos['user_id'].unique())
unique_users

for user in unique_users:
    try:
        repo_ids = get_recomm_repo_ids(model, user, 50)
        repo_recs = get_recomm_repo_urls(users_repos, repo_ids)
        username = (users_repos.loc[users_repos['user_id'] == user, 'login']).unique()[0]
        repo_recs['recs_for_userid'] = user
        repo_recs['recs_for_userlogin'] = username
        repo_recs['rank'] = [num for num in range(1,51)]
        repo_recs.to_sql(name='all_top50_repo_recs', 
                        con=engine, 
                        if_exists='append',
                        chunksize=10000,
                        index=False)
    except:
        num_users_not_in_model += 1
        continue

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [ ]:
# THIS WORKED
repo_rec = pd.DataFrame({'test': [num for num in range(1,200001)], 'test2': [num for num in range(1,200001)]})

repo_rec.to_sql(name='test', 
                con=engine, 
                if_exists='append',
                chunksize=10000,
                index=False)

In [ ]:

CREATE TABLE all_top50_repo_recs
(
    user_id int,
    repo_id int,
    log10_commits int
);

In [ ]:
mysqlimport --ignore-lines=1 \
            --fields-terminated-by=, \
            --local -u root \
            -p user_recs \
             entire_commits.csv